### Use case notebook: schedule CCB team meetings ###

In [1]:
import os
import glob
import numpy as np

# There is a warning in the timeboard library that we want to suppress here
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
import time
import timeboard as tb
import timeboard.calendars.US as US
import datetime
import holidays

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the package
%load_ext autoreload
%autoreload 2
import cadence
from cadence.utils import FileOP
from cadence.mscheduler import Meetings
print(f'Package version: {cadence.__version__}')

Package version: unknown


### Meeting participants and settings ###

In [2]:
# Local data
data_dir = os.path.join(os.environ.get('HOME'), 'data')
group_members_file_name = 'ccb_names_list.csv'
link = os.path.join('https://dsets.s3.amazonaws.com', group_members_file_name)
team = FileOP().download_from_url(url=link, download_dir=data_dir)
tm_raw = pd.read_csv(team)
display(tm_raw)

ccb_names_list.csv: 8.19kB [00:00, 49.2kB/s]

Created .csv file.


,name,email,group
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data
2,Nathan,nathan_palmer@hms.harvard.edu,ai-data
3,Sam,samantha_pullman@hms.harvard.edu,ai-data
4,Sreya,sreya_banerjee@hms.harvard.edu,ai-data
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio
7,Grey,grey_kuling@hms.harvard.edu,comp-bio
8,Ludwig,ludwig_geistlinger@hms.harvard.edu,comp-bio
9,Tram,tram_nguyen@hms.harvard.edu,comp-bio


### Cleanup of the spreadsheet ###
Just in case that was not done. To provide a consistent look.

In [4]:
email_col = 'email'
name_col = 'name'
group_col = 'group'

# Lets do some cleanup of the spreadsheet
# small letters for the column names
tm = tm_raw.copy()
tm.columns = [col.lower() for col in tm.columns]

# small letters for emails and group, first letter capitalize for names
tm[email_col] = tm[email_col].str.lower()
tm[group_col] = tm[group_col].str.lower()
tm[name_col] = tm[name_col].str.title()

# Sort by group and then name
tm = tm.sort_values(by=['group', 'name'], ascending=True).reset_index(drop=True)

# Mark names that are not presenting
rm_name = ['nathan', 'ludwig']
rm_name = [nm.title() for nm in rm_name]
tm = tm.assign(presenting=True)
tm.loc[tm['name'].isin(rm_name), 'presenting'] = False

# save the name ist as a csv_file
presenter_list_file_name = f'{os.path.splitext(group_members_file_name)[0]}_list.csv'
presenter_list_file = os.path.join(data_dir, presenter_list_file_name)
tm.to_csv(presenter_list_file, index=False)
display(tm)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data,True
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,True
2,Nathan,nathan_palmer@hms.harvard.edu,ai-data,False
3,Sam,samantha_pullman@hms.harvard.edu,ai-data,True
4,Sreya,sreya_banerjee@hms.harvard.edu,ai-data,True
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,True
7,Grey,grey_kuling@hms.harvard.edu,comp-bio,True
8,Ludwig,ludwig_geistlinger@hms.harvard.edu,comp-bio,False
9,Tram,tram_nguyen@hms.harvard.edu,comp-bio,True


In [5]:
# Remove the names that we do not want in the schedule
presenters_df = tm.loc[tm['presenting']==True]
display(presenters_df)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data,True
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,True
3,Sam,samantha_pullman@hms.harvard.edu,ai-data,True
4,Sreya,sreya_banerjee@hms.harvard.edu,ai-data,True
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,True
7,Grey,grey_kuling@hms.harvard.edu,comp-bio,True
9,Tram,tram_nguyen@hms.harvard.edu,comp-bio,True
10,Tyrone,tyrone_lee@hms.harvard.edu,comp-bio,True


### Shuffle the data and create a presenter list ###

In [6]:
# Let's do a random shuffling of the data frame
random_state = 984
presenters_df_shuffled = presenters_df.sample(frac=1, random_state=random_state)

# Get the names and the groups from the new data frame
name_list = list(presenters_df['name'].values)
group_list = list(presenters_df['group'].values)

# Instantiate the Meetings class with the list of names and groups from the data frame
meet = Meetings(name_list=name_list, group_list=group_list)

# We create a presenter list by merging the groups so that we have a member from a different group presenting each time
presenter_list = meet.create_name_sequence()
print(presenter_list)
display(meet.name_df)

['Andreas', 'Andrew', 'Gerald', 'Anthony', 'Sam', 'Grey', 'Sreya', 'Tram', 'Tyrone']


,name,group
0,Andreas,ai-data
1,Andrew,comp-bio
2,Gerald,ai-data
3,Anthony,comp-bio
4,Sam,ai-data
5,Grey,comp-bio
6,Sreya,ai-data
7,Tram,comp-bio
8,Tyrone,comp-bio


### Manipulate the sequence of presenters ###

In [7]:
presenter_list = list(meet.name_df['name'].values)
print(presenter_list)

# If we don't like this sequence, we can create a new one
new_presenter_list = ['Gerald', 'Grey', 'Sam', 'Tyrone', 'Sreya', 'Tram', 'Andreas', 'Anthony', 'Andrew']
print(new_presenter_list)

# We can replace the list of presenters with our new one. Just make sure that the names match.
new_presenter_list = meet.create_name_sequence(name_sequence=new_presenter_list, merge_groups=False)
print(new_presenter_list)

# We can see, that the internal data frame with the group associations has changed according to the new list
print()
display(meet.name_df)

['Andreas', 'Andrew', 'Gerald', 'Anthony', 'Sam', 'Grey', 'Sreya', 'Tram', 'Tyrone']
['Gerald', 'Grey', 'Sam', 'Tyrone', 'Sreya', 'Tram', 'Andreas', 'Anthony', 'Andrew']
['Gerald', 'Grey', 'Sam', 'Tyrone', 'Sreya', 'Tram', 'Andreas', 'Anthony', 'Andrew']



,name,group
0,Gerald,ai-data
1,Grey,comp-bio
2,Sam,ai-data
3,Tyrone,comp-bio
4,Sreya,ai-data
5,Tram,comp-bio
6,Andreas,ai-data
7,Anthony,comp-bio
8,Andrew,comp-bio


### Create the meeting schedule ###

In [11]:
start_date = '2024-8-21'
end_date = '2024-12-31'

cal = meet.create_timeboard(start_date=start_date, 
                            end_date=end_date,
                            start_name='Andrew')

skip_date = '2024-08-28'
skip_name = 'Everyone'
skip_comment = 'Repos, best practices discussion'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

skip_date = '2024-10-09'
skip_name = 'Nathan'
skip_comment = ''

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

skip_date = '2024-10-23'
skip_name = 'Devesh Surve'
skip_comment = 'Candidate Presentation'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

skip_date = '2024-11-06'
skip_name = 'No meeting'
skip_comment = 'No meeting'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

skip_date = '2024-11-13'
skip_name = 'No meeting'
skip_comment = 'No meeting'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)


skip_date = '2024-11-27'
skip_name = 'Happy Thanksgiving'
skip_comment = 'Happy Thanksgiving!'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

display(cal)

,date,name,group,holiday,comment
0,2024-08-21,Andrew,comp-bio,False,None
1,2024-08-28,Everyone,ai-data,False,"Repos, best practices discussion"
2,2024-09-04,Gerald,ai-data,False,None
3,2024-09-11,Grey,comp-bio,False,None
4,2024-09-18,Sam,ai-data,False,None
5,2024-09-25,Tyrone,comp-bio,False,None
6,2024-10-02,Sreya,ai-data,False,None
7,2024-10-09,Nathan,comp-bio,False,
8,2024-10-16,Tram,comp-bio,False,None
9,2024-10-23,Devesh Surve,ai-data,False,Candidate Presentation


In [13]:
# Save the presentation schedule
save_cols = ['date', 'name', 'comment']
schedule_name = f'ccb_presentations_2024_4.csv'
schedule_file = os.path.join(data_dir, schedule_name)
cal[save_cols].to_csv(schedule_file, index=False)
display(cal[save_cols])
print(schedule_file)

,date,name,comment
0,2024-08-21,Andrew,None
1,2024-08-28,Everyone,"Repos, best practices discussion"
2,2024-09-04,Gerald,None
3,2024-09-11,Grey,None
4,2024-09-18,Sam,None
5,2024-09-25,Tyrone,None
6,2024-10-02,Sreya,None
7,2024-10-09,Nathan,
8,2024-10-16,Tram,None
9,2024-10-23,Devesh Surve,Candidate Presentation


/app/data/ccb_presentations_2024_4.csv
